In [ ]:
from openai import OpenAI
OPEN_API_KEY = ""
client = OpenAI(api_key=OPEN_API_KEY)

In [ ]:
import re
import json
import pandas as pd
import openai
import string
import requests
import random

In [ ]:
# if mounting to drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
prompts = pd.read_csv("weat_associations_dynamic_prompt.csv")

In [ ]:
# all effect sizes > 0 are considered female associated
female = prompts[prompts['effect_size'] > 0]
male = prompts[prompts['effect_size'] < 0]

In [ ]:
neutral = pd.concat([female.sample(40), male.sample(40)])
neutral = neutral.sample(frac=1)

In [ ]:
tasks = list(female["Task"])
output = list(female["Output"])

fem_data = []
for i in range(len(tasks)):
  instance = {"messages": [{"role": "user", "content": tasks[i]}, {"role": "assistant", "content": output[i]}]}
  fem_data.append(instance)

In [ ]:
output_file_path = "female_associated_instances.jsonl"
with open(output_file_path, "w") as json_file:
    # Write the list of dictionaries to the JSON file
    for i in fem_data:
      json.dump(i, json_file)
      json_file.write("\n")

In [ ]:
tasks = list(male["Task"])
output = list(male["Output"])

male_data = []
for i in range(len(tasks)):
  instance = {"messages": [{"role": "user", "content": tasks[i]}, {"role": "assistant", "content": output[i]}]}
  male_data.append(instance)

In [ ]:
output_file_path = "male_associated_instances.jsonl"
with open(output_file_path, "w") as json_file:
    # Write the list of dictionaries to the JSON file
    for i in male_data:
      json.dump(i, json_file)
      json_file.write("\n")

In [ ]:
tasks = list(neutral["Task"])
output = list(neutral["Output"])

neu_data = []
for i in range(len(tasks)):
  instance = {"messages": [{"role": "user", "content": tasks[i]}, {"role": "assistant", "content": output[i]}]}
  neu_data.append(instance)

In [ ]:
output_file_path = "neutral_associated_instances.jsonl"
with open(output_file_path, "w") as json_file:
    # Write the list of dictionaries to the JSON file
    for i in neu_data:
      json.dump(i, json_file)
      json_file.write("\n")

In [ ]:
client.files.create(
  file=open("neutral_associated_instances.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-uRnV1ZQL4B46YTCnRGf3YyIC', bytes=11894, created_at=1714594487, filename='neutral_associated_instances.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.fine_tuning.jobs.create(
  training_file='file-uRnV1ZQL4B46YTCnRGf3YyIC',
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-QEWoVemrBtbFg76hdMCoTYLJ', created_at=1714594495, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-lvAMHnNWT1zs6tJUQoOSkpSF', result_files=[], seed=670407658, status='validating_files', trained_tokens=None, training_file='file-uRnV1ZQL4B46YTCnRGf3YyIC', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)